In [ ]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
from dash import Dash, html, dcc

# Sample data
data = {
    'A': [1, 2, 3, 2.5],
    'B': [4, 1, 5, 3.0],
    'C': [2, 3, 1, 2.0]
}

# Create DataFrame
df = pd.DataFrame.from_dict(data, orient='index', columns=['col1', 'col2', 'col3', 'col4'])

# Calculate deviations from col4 (row-wise)
diff = df[['col1', 'col2', 'col3']].subtract(df['col4'], axis=0)

# Add col4 with zero deviation (acts as neutral gray)
diff['col4'] = 0
diff = diff[['col1', 'col2', 'col3', 'col4']]

# Original values as text
text = df[['col1', 'col2', 'col3', 'col4']]

# Diverging color scale
colorscale = 'RdYlGn'

# Normalize color range
zmin = diff.values.min()
zmax = diff.values.max()
absmax = max(abs(zmin), abs(zmax))

# Build heatmap
fig = go.Figure(data=go.Heatmap(
    z=diff.values,
    x=diff.columns,
    y=diff.index,
    colorscale=colorscale,
    zmin=-absmax,
    zmax=absmax,
    colorbar=dict(title='Deviation'),
    text=text.values,
    texttemplate="%{text}",
    hovertemplate='Metric: %{x}<br>Index: %{y}<br>Value: %{text}<br>Diff from col4: %{z:.2f}<extra></extra>'
))

# Layout customization
fig.update_layout(
    title='Deviation from col4 (Green = Higher, Red = Lower)',
    xaxis_title='Metrics',
    yaxis_title='Index',
    width=1000,
    height=600,
    font=dict(size=18)
)

# Dash app
app = Dash(__name__)

app.layout = html.Div([
    html.H1("Metric Heatmap vs col4"),
    dcc.Graph(figure=fig)
])

# Run app
if __name__ == '__main__':
    app.run_server(debug=True)
